In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from scipy.spatial import cKDTree
from shapely.geometry import Point
from preparacion_datos.src.preparacion_inter_puntos import prep
import glob
import tqdm as tqdm
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.geometry
import warnings
from shapely.geometry import shape
warnings.filterwarnings("ignore")
from puntos_sobre_poligono.simulacion_interpolacion import task_chunks

### PREDIAL

In [ ]:
casas=gpd.read_file(r'C:\Users\dlara\Naucalpan_new_casas.shp')
# terrenos=gpd.read_file(r'C:\Users\dlara\Downloads\naucalpancompleto\new_terreno1.shp')

In [ ]:
predial=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PREDIAL (16022022).xlsx')
adeudo=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PREDIALADEUDO (16022022).xlsx')
propietario=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PROPIETARIO (16022022).xlsx')

In [ ]:
base_pred= pd.read_csv(r'C:\Users\dlara\Downloads\Predial.csv')

In [ ]:
from geoloc2.Valor_Catastral.calculo_predial import *

In [ ]:
curts= gpd.read_file(r'C:\Users\dlara\Downloads\098 Naucalpan\NAUCALPAN_PREDIOS.shp')

In [ ]:
NAUC=pd.read_csv(r'C:\Users\dlara\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv', encoding='utf-8-sig')

In [ ]:
def transform_df_to_gpd(df: pd.DataFrame,
                    lon_col: str = 'INTFIS_RS_FD_LON_1',
                    lat_col: str = 'INTFIS_RS_FD_LAT_1', crs= 'EPSG:4326'):
        '''Transforma base a partir de dos columnas de latitud y longitud'''
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(
                df[lon_col], df[lat_col]
            ),
            crs=crs,
        )
        return gdf

In [ ]:
NAUC['INTFIS_RS_FD_LAT']= NAUC['INTFIS_RS_FD_LAT_1'].combine_first(NAUC['INTFIS_RS_FD_LAT'])
NAUC['INTFIS_RS_FD_LON']= NAUC['INTFIS_RS_FD_LON_1'].combine_first(NAUC['INTFIS_RS_FD_LON'])

In [ ]:
NAUC=transform_df_to_gpd(NAUC, lon_col= 'INTFIS_RS_FD_LON',lat_col = 'INTFIS_RS_FD_LAT')

In [ ]:
PRED= NAUC.loc[NAUC['CLAVECATASTRAL'].notna()]

In [ ]:
Manzanas=gpd.read_file(r'C:\Users\dlara\Downloads\Full manzanas\Full manzanas\Manzana_Naucalpan.shp')

In [ ]:
Manzanas['geometry']=Manzanas.buffer(100)

In [ ]:
Manz=Manzanas.dissolve()

In [ ]:
NAUC=NAUC.to_crs(Manz.crs)
def new_func(NAUC):
    return gpd.sjoin(NAUC,Manz).drop(columns=['index_right',	'mun',	'zona',	'manz',	'cve_cat'])

rectificado_nauc=new_func(NAUC)

In [ ]:
curts=curts.to_crs(Manz.crs)
curts_rectificado=gpd.sjoin(curts,Manz).drop(columns=['index_right',	'mun',	'zona',	'manz',	'cve_cat'])

In [ ]:
casas=casas.to_crs(Manz.crs)
casas['ID']=casas.reset_index()['index']
casas_rectificado=gpd.sjoin(casas,Manz).drop(columns=[	'index_right',	'mun',	'zona',	'manz',	'cve_cat'])

## Cruces

In [ ]:

    ARR=linestring.geometry.astype(str)
    # print(ARR)
    if len(ARR)>0:
        # print(ARR)
        
        # print(l1)
            try:
                # print(ARR)
                l1=ARR.str.rfind('POLYGON')
                l1=int(l1)
                # print(ARR)
                list_wkt = ARR.str.replace("GEOMETRYCOLLECTION (",'', regex=False).str.replace("LINESTRING ",'POLYGON ', regex=False)#.str.replace("POLYGON ",'', regex=False).str.replace("LINESTRING ",'', regex=False).str.replace('((','[', regex=False).str.replace('))',']', regex=False)
                list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
                multi=shapely.geometry.MultiPolygon(list_polygons)
                # print(multi)
                # print('multi', multi)
                linestring['geometry'] = multi


            except:
                # print(ARR)
                l1=ARR.str.find('LINESTRING')
                print(ARR.str[:l1])
                list_wkt = ARR.str.replace(ARR.str[:l1],'').str.replace("GEOMETRYCOLLECTION (",'').str.replace("LINESTRING ",'POLYGON ', regex=False)
                list_wkt= [list_wkt+str('))')]
                # print(list_wkt)
                list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
                multi=shapely.geometry.MultiPolygon(list_polygons)
                # print('multi', multi)
                linestring['geometry'] = multi
                
            for col in dictt['cata']:
                try:
                    
                    BASE=BASE[~BASE[col].isin(linestring[col])]
                except:
                    pass
    return(BASE)
            

In [ ]:
any(linestring.columns) == dictt['cata'][0]

In [ ]:
def line2poli(BASE):    
    linestring=BASE.loc[(BASE["geometry"].map(lambda y: isinstance(y,MultiPolygon))==False)&(BASE["geometry"].map(lambda y: isinstance(y,Polygon))==False)]
    # print(BASE.plot(), '1')
    linestring=linestring.explode()
    
    newpols=linestring.loc[(linestring["geometry"].map(lambda y: isinstance(y,MultiPolygon))==True)|(linestring["geometry"].map(lambda y: isinstance(y,Polygon))==True)]
    # print(newpols.shape)
    BASES=gpd.GeoDataFrame()
    for col in dictt['cata']:
        try:
            BASE=BASE[~BASE[col].isin(linestring[col])]
            # print(BASE.shape)
            BASES=pd.concat([BASES, BASE])
            # print(BASE.shape)
            # print(BASE.plot(), '2')
            # print(BASE)
        except:
            continue
       
        # print(final.shape)
    print(BASES.plot())
    return(pd.concat([BASES,newpols]))

In [ ]:
linestring=shps_merged.loc[(shps_merged["geometry"].map(lambda y: isinstance(y,MultiPolygon))==False)&(shps_merged["geometry"].map(lambda y: isinstance(y,Polygon))==False)].dissolve()

ARR=linestring.geometry.astype(str)
print(ARR)

if len(ARR)>0:
    # print(ARR)
    
    # print(l1)
    for i in range(len(ARR)):
        try:
            # print(ARR)
            l1=ARR.str.find('POLYGON')
            # print(ARR.str.replace("GEOMETRYCOLLECTION (POLYGON ",''))
            list_wkt = ARR.str.replace("GEOMETRYCOLLECTION (POLYGON ",'')
            # print(ARR,list_wkt)
            list_wkt= [list_wkt+str('))')]
            # print(list_wkt)
            list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
            multi=shapely.geometry.MultiPolygon(list_polygons)
            # print(multi)
            # print('multi', multi)
            linestring['geometry'] = multi

        except:
            # print(ARR)
            l1=ARR.str.find('LINESTRING')
            # print(ARR.str[:l1])
            list_wkt = ARR.str.replace(ARR.str[:l1],'').str.replace("GEOMETRYCOLLECTION (LINESTRING ",'')
            list_wkt= [list_wkt+str('))')]
            # print(list_wkt)
            list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]
            multi=shapely.geometry.MultiPolygon(list_polygons)
            # print('multi', multi)
            linestring['geometry'] = multi
            
        for col in dictt['cata']:
            try:
                
                shps_merged=shps_merged[~shps_merged[col].isin(linestring[col])]
            except:
                pass
        
        shps_merged = pd.concat([shps_merged,linestring])

In [ ]:
dictt= {'cata':['id_cat','cve_cat','CLAVECATAS','CLAVECATASTRAL_1','CLAVECATASTRAL_2']}\n",
dictt['cata']

In [ ]:
path=r"C:\Users\dlara\Desktop\MUNICIPIOS\Insumos_municipios_en_proceso\*"
filenames= glob.glob(path)
for files in tqdm.tqdm(filenames):

    shapes={}
       
    i=0
    files=files.replace('\\',"/")
    path2=files.replace('\\',"/").replace('/*',"/")
    
    for file in glob.glob(path2 + '/*.shp'):
        file= file.replace('/',"\\")
        name=file.replace(path2.replace('/',"\\"),'').replace('.shp',"").replace('\\',"") 
        i=i+1
        Nname= name + str(i)
    #   
        shapes[Nname]=gpd.read_file(file)
        
        # print(Nname)
        # reemplazar id cat por clave catastral
        
        for col in dictt['cata']:
            shapes[Nname].rename(columns={col:'CLAVECATASTRAL'}, inplace=True)
        #Disolución de las geometrías a partir de clavecatastral 
        shapes[Nname]=shapes[Nname].dissolve(by='CLAVECATASTRAL').reset_index()
        
        if shapes[Nname].crs == 4326:
            pass
        else:
            shapes[Nname]=shapes[Nname].to_crs(4326)

        lista= list(shapes.keys())  
        i=0     
        shps_merged= shapes[lista[i]]
        shps_merged.crs= 4326
        if len(shapes)>1:
            i=1
            shps_merged=shapes[lista[i]].overlay(shps_merged, how='union',keep_geom_type=False)
                # print(shps_merged)
            # try:
                # print(shps_merged.plot(),'1')
                
            shps_merged= line2poli(shps_merged)
            # print(shps_merged.plot(),'2')
            for i in shps_merged.columns:
                if i != 'geometry':
                    shps_merged[i]=shps_merged[i].astype(str)
            shps_merged.reset_index(drop=True, inplace=True)
            shps_merged.set_geometry('geometry').to_file(r'C:/Users/dlara/Desktop/MUNICIPIOS/IGECEM/' + Nname + "_cruce_shapes.shp")

            # except:
            #     print('except')
            
                # shps_merged= shapes[lista[0]]
                # shps_merged=shps_merged.overlay(shapes[lista[i]], how='union',keep_geom_type=False)
                
                # shps_merged= line2poli(shps_merged)
                # print(name)
                # print(shps_merged)
        else:
            print('else')
            
            continue
        
        # shps_merged.reset_index(drop=True, inplace=True)
        # print(r'C:/Users/dlara/Desktop/MUNICIPIOS/IGECEM/' + Nname +  str(i) + "_cruce_shapes.shp")
        
        # shps_merged.to_file(r'C:/Users/dlara/Desktop/MUNICIPIOS/IGECEM/' + Nname +  str(i) + "_cruce_shapes.shp")
                # else: 
                # print('error')  


In [ ]:
full_igecem=gpd.read_file(r'C:\Users\dlara\Desktop\MUNICIPIOS\IGECEM\predios_Ixtapan_Sal.1_cruce_shapes.shp')
full_igecem.plot()

In [ ]:
PRED=pd.read_csv(r'C:\Users\dlara\Atlacomulco_puntos.csv', encoding='utf-8-sig')
PRED['CLAVECATASTRAL']=PRED['CLAVECATASTRAL'].astype(str).str.replace('.0','', regex=False).str.zfill(16)
# 
PRED[['CLAVECATASTRAL', 'LONGITUD','LATITUD','CURT_1', 'curt','geometry','min_dist']].dropna()

# prep.transform_df_to_gpd(PRED, 'LONGITUD_1', 'LATITUD_1', 3857).to_crs(4326)

PRED.drop_duplicates('geometry', inplace=True)
PRED.dropna(axis=1, inplace=True)
puntos=prep.transform_df_to_gpd(PRED, 'LONGITUD_1', 'LATITUD_1', 3857)
puntos.dropna(axis=1, inplace=True)
rev_pred=gpd.read_file(r'C:\Users\dlara\Downloads\final\final\test_igecem_final.shp')

UNICOS=rev_pred.loc[rev_pred['CURT_1']==1]
nounicos=rev_pred.loc[rev_pred['CURT_1'].fillna(0)!=1]
nounicos=nounicos.to_crs(3857)
puntos=puntos.set_geometry('geometry')
rev_pred=rev_pred.set_geometry('geometry')
rev_pred.drop_duplicates('CLAVECATAS', keep='first', inplace=True)
# from preparacion_datos.src.preparacion_inter_puntos import ckdnearest

j_puntos=prep.ckdnearest(puntos, puntos[['geometry']])
cruce_check= gpd.sjoin(nounicos.to_crs(3857),j_puntos.loc[j_puntos['min_dist_2']>30][['geometry', 'LONGITUD_1', 'LATITUD_1','min_dist']], how='right')

cruce_check.to_file('C:/Users/dlara/Atlacomulco_predial_final_curts_reps.shp')
UNICOS.to_file('C:/Users/dlara/Atlacomulco_predial_final_curts_unicos.shp')

## Cruce_final

In [170]:
def deduplicate(geo_data: np.ndarray # shape == (N, 4)
        ) -> np.ndarray:             # deduplicated data with origin order
    data = geo_data.reshape(-1, 2, 2)
    dt = f'f{data.itemsize}' # f4 or f8
    data = data.view([('x', dt), ('y', dt)]) 
    # eliminate differences
    ixs = np.argsort(data, -2, order=('x', 'y'))
    data_no_df = np.take_along_axis(data, ixs, axis=-2) # sorted by 'x' then by 'y'
    # get unique
    unique_sorted_data, uni_ixs = np.unique(data_no_df, True, axis=0)
    uni_ixs.sort() # inplace sort 1d-array
    data_deduplicated = geo_data[uni_ixs] # unique, originally ordered and shaped
    return data_deduplicated

In [ ]:
points= cruce_check.geometry
geoms = [shape(feat["geometry"]) for feat in points ]
list_arrays = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms ]
result = deduplicate(list_arrays)
final_result = [list(map(tuple, pair)) for pair in result.tolist()]
pnts = [Point(pair) for pair in final_result]
# cruce_check['geometry'] = pnts
geometria=gpd.GeoSeries(pnts)

In [2]:
m1= pd.read_csv(r'C:\Users\dlara\Desktop\MUNICIPIOS\Salidas\Ixtapan_Base_predial_PREGEO.csv', encoding='utf-8-sig')
rev_pred= gpd.read_file(r'C:\Users\dlara\Desktop\MUNICIPIOS\Ixtapan.shp').to_crs(3857)

In [3]:
m1= prep.transform_df_to_gpd(m1, 'LONGITUD','LATITUD', 4326)
N1=m1.loc[(m1['LATITUD']==18)| (m1['LATITUD'].isna())]
N2=m1.loc[(m1['LATITUD']!=18)| (m1['LATITUD'].isna())]

In [4]:
rev_pred['CLAVE_CATASTRAL']=rev_pred['cve_cat'].str[:-8]
N1['CLAVE_CATASTRAL']=N1['CLAVE_CATASTRAL'].astype(str).str.zfill(10).str[:-2]

In [5]:
N1.drop(columns='geometry', inplace=True)

In [6]:
nm2=gpd.sjoin(rev_pred[['cve_cat','geometry']],N2.to_crs(3857).drop(columns='cve_cat'))
nm1=pd.merge(N1.drop(columns='cve_cat'),rev_pred[['cve_cat','geometry','CLAVE_CATASTRAL']], on='CLAVE_CATASTRAL')

In [184]:
filenames= glob.glob(r"C:\Users\dlara\Desktop\MUNICIPIOS\*.csv")

for path in tqdm.tqdm(filenames):
    # filename= glob.glob(path)
    puntos=pd.read_csv(path)
    try:
        puntos= prep.transform_df_to_gpd(puntos, 'LONGITUD_1','LATITUD_1', 3857)
    except:
        puntos= prep.transform_df_to_gpd(puntos, 'LONGITUD','LATITUD', 4326).to_crs(3857)
    rev_pred=gpd.read_file(path[:path.find('_puntos.csv')] + '.shp')
    rev_pred=rev_pred.to_crs(3857)
    puntos=puntos.set_geometry('geometry')
    rev_pred=rev_pred.set_geometry('geometry')
    # puntos.drop_duplicates('CLAVECATASTRAL', keep='first', inplace=True)
    puntos=puntos.groupby('geometry', sort=False, dropna=True).first().reset_index()
    try:
        UNICOS=puntos.loc[puntos['CURT_1']<=1]
        
        # UNICOS['geometry']=UNICOS.geometry.centroid
        nounicos=puntos.loc[puntos['CURT_1']>1]
    except:
        UNICOS=puntos.loc[puntos['CURT']<=1]
        nounicos=puntos.loc[puntos['CURT']>1]
    j_puntos=prep.ckdnearest(nounicos, nounicos[['geometry']], k=2)
    # print(nounicos, 'no unicos')
    # print(nounicos.crs, 'no unicos crs')
    # print(j_puntos, 'puntos')
    # print(j_puntos.crs,'puntos crs')
    # print(gpd.sjoin(nounicos,j_puntos.loc[j_puntos['min_dist_2']==0][['geometry', 'LONGITUD', 'LATITUD']]))
    try:
        cruce_check= gpd.sjoin(j_puntos.loc[j_puntos['min_dist_2']==0][['geometry', 'LONGITUD_1', 'LATITUD_1']],rev_pred, how='left')
    except:
       cruce_check= gpd.sjoin(j_puntos.loc[j_puntos['min_dist_2']==0][['geometry', 'LONGITUD', 'LATITUD']],rev_pred,how='left') 
    # cruce_check=cruce_check.loc[cruce_check['CLAVECATASTRAL'].notna()]
    cruce_check=cruce_check.groupby('geometry', as_index=False,sort=False).first().reset_index(drop=True)
#   cruce_check.loc[cruce_check['CLAVECATAS']=='0240127004000000'].plot()
    file_name='Salidas'
    name=path[path.rfind('\\')+1:path.rfind('_puntos')]
    path12= os.path.join(path[:path.rfind('MUNICIPIOS')],'MUNICIPIOS' ,file_name)
    # cruce_check.to_file(path12 + '\\' +name+  '_predial_final_curts_reps.shp')
    # UNICOS.to_file(path12 + '\\' +name+ '_predial_final_curts_unicos.shp')
    # final_pred= pd.concat([cruce_check, UNICOS])
   
    filename2= glob.glob(r"C:\Users\dlara\Desktop\MUNICIPIOS\Entradas\*.shp")
    for path2 in filename2:
        
        if 'Ixtapan' in path2:
            
            cruces_ads=gpd.read_file(path2)
            print(cruces_ads.crs)
            name2=path2[path2.rfind('\\'+name)+1:path2.rfind('_cruces_todos')]
        else:
            print('El archivo aun no se encuentra en la carpeta')
            break
        # if cruces_ads.crs== 4326:
        #     cruces_ads=cruces_ads.to_crs(3857)
        # elif cruces_ads.crs == 3857:
        #     continue           
        # print(cruces_ads.columns)
        ### el primero es fácil porque es directo, resultado: poli 
        # gato   
        drop=UNICOS.columns[(UNICOS.columns.str.contains('left|right'))]
        UNICOS.drop(columns=drop, inplace=True)
        drop=cruces_ads.columns[(cruces_ads.columns.str.contains('left|right'))]
        cruces_ads.drop(columns=drop, inplace=True)
        primero=gpd.sjoin(cruces_ads,UNICOS, how='left' )
        
        ## diferenciando
        temp1=cruces_ads.loc[~cruces_ads['CURT_f'].isin(primero['CURT_f'])]
        ##cruce
        drop=temp1.columns[(temp1.columns.str.contains('left|right'))]
        temp1.drop(columns=drop, inplace=True)
        drop=cruce_check.columns[(cruce_check.columns.str.contains('left|right'))]
        cruce_check.drop(columns=drop, inplace=True)
        segundo= gpd.sjoin(temp1,cruce_check,how='left')
    
        sobra_pred=cruce_check[~cruce_check.geometry.isin(segundo.geometry)]
        sobra_pred.geometry=sobra_pred.geometry.buffer(2)
        real_ads= temp1.loc[~temp1['CURT_f'].isin(segundo['CURT_f'])]
        sobra_pred['CAT_1']='FUERA'
        primero['CAT_1']='PREDIAL_UNICOS'
        segundo['CAT_1']= 'PREDIAL_REPETIDOS'
        pred_final= pd.concat([primero, segundo,sobra_pred])
        pred_final.dropna(how='all', axis=1, inplace=True)
        drop=pred_final.columns[(pred_final.columns.str.contains('0|TYPES|TYPE|ADDRESS|PLUS_CODE|_GEO_|Unnamed'))]
        pred_final.drop(columns=drop, inplace=True)
        drop2=pred_final.columns[(pred_final.columns.str.contains(r'[()\"#/@;:<>{}`+=~|.!?,]'))]
        pred_final.drop(columns=drop2, inplace=True)
        # pred_final.columns=pred_final.columns.str.replace(r"[()\"#/@;:<>{}`+=~|.!?,]", '')
        pred_final.rename(columns={'USO DE ÁREA HOMOGÉNEA':'USO DE AREA HOMOGENEA'}, inplace=True)
        for i in pred_final.columns:
            if i != 'geometry':
                pred_final.loc[pred_final[i].astype(str).str.contains(r'[^A-Za-z0-9]+'),i]=pred_final.loc[pred_final[i].astype(str).str.contains(r'[^A-Za-z0-9]+')][i].astype(str).str.replace(r'[^A-Za-z0-9]+','')
        gpd.GeoDataFrame(pred_final, geometry='geometry', crs=3857).to_file(path12 + '/' +name2+ 'base_final_predial_corregida.shp')
        real_ads['CAT_1']= 'ADICIONALES'
        base_final=pd.concat([pred_final, real_ads])
        base_final.columns=base_final.columns.str.replace(r"[()\"#/@;:<>{}`+=~|.!?,]", '')
        drop2=base_final.columns[(base_final.columns.str.contains(r'[()\"#/@;:<>{}`+=~|.!?,]'))]
        base_final.drop(columns=drop2, inplace=True)
        base_final.rename(columns={'USO DE ÁREA HOMOGÉNEA':'USO DE AREA HOMOGENEA'}, inplace=True)
        for i in base_final.columns:
            if i != 'geometry':
                base_final.loc[base_final[i].astype(str).str.contains(r'[^A-Za-z0-9]+')][i]=base_final.loc[base_final[i].astype(str).str.contains(r'[^A-Za-z0-9]+')][i].astype(str).str.replace(r'[^A-Za-z0-9]+','')
        
        gpd.GeoDataFrame(base_final, geometry='geometry', crs=3857).to_file(path12 + '/' +name2+ 'base_final_ads_pred.shp')


  0%|          | 0/2 [00:00<?, ?it/s]

epsg:3857


Failed to create file C:\Users\dlara\Desktop\MUNICIPIOS\Salidas/C:\Users\dlara\Desktop\MUNICIPIOS\Entradas\Ixtapan_casasbase_final_predial_corregida.shp: No error
  0%|          | 0/2 [01:01<?, ?it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "fiona\ogrext.pyx", line 1133, in fiona.ogrext.WritingSession.start
  File "fiona\_err.pyx", line 291, in fiona._err.exc_wrap_pointer
fiona._err.CPLE_AppDefinedError: Failed to create file C:\Users\dlara\Desktop\MUNICIPIOS\Salidas/C:\Users\dlara\Desktop\MUNICIPIOS\Entradas\Ixtapan_casasbase_final_predial_corregida.shp: No error

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\dlara\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\dlara\AppData\Local\Temp\ipykernel_16724\365799180.py", line 94, in <module>
    gpd.GeoDataFrame(pred_final, geometry='geometry', crs=3857).to_file(path12 + '/' +name2+ 'base_final_predial_corregida.shp')
  File "c:\Users\dlara\anaconda3\lib\site-packages\geopandas\geodataframe.py", line 1114, in to_file
    _to_file(self,

In [ ]:
def post_points_catastro(path_base, path_shp, funcion):
    """
        De la informacion que se obtiene, separa la lat y la lon, lo limpia y concatena, quita distancia menor entre ambos parametros.
        Se genera una base total, se hace un proceso de diatancia minima para quitar los puntos que cruzan entre si.

        Los chunks se obtienen a partir de los cores de la pc en que se ejecute este script
    """
    if path_base == float('Nan'):
        test_igecem = prep.data_prep_catastro(path_base, path_shp)
        # print('test igecem es: ',test_igecem)
    else:
        m_igecem = gpd.read_file(path_shp) ##Lee desde shp
        m_igecem.crs= 4326 #m_igecem.to_crs(4326)
    try:
        m_igecem = m_igecem.loc[~m_igecem['manz'].astype(str).str.endswith('000')]
    except: 
        pass
    test_igecem = m_igecem
    prep.replace_columns(test_igecem)
    test_igecem.drop(columns= test_igecem.columns[(test_igecem.columns.str.contains('index'))|(test_igecem.columns.duplicated())], inplace=True)
    # print('test_igecem:  ',test_igecem.columns[test_igecem.columns.duplicated()])
    # test_igecem_chunks =  np.array_split(test_igecem, 1) ##Aqui se especifica si se requiere un loc y los chunks
    
    # df_concat = pd.DataFrame()
    
    # with concurrent.futures.ThreadPoolExecutor() as executor:
    #     for df_chunk in tqdm(zip(executor.map(funcion, test_igecem_chunks)),total = len(test_igecem_chunks)):
    #         df_concat = pd.concat([df_chunk[0],df_concat], axis=0)

    return df_concat

In [171]:
ads_final= gpd.read_file(r'C:\Users\dlara\Downloads\Atlacomulco_predial_Z\Atlacomulco_predial_Z.shp')

### Predial con simulación

In [188]:
pred.columns

Index(['DIRECCION', 'VALORTERRENO', 'VALORTERRENOCOMUN', 'VALORCONSTPROP',
       'VALORCONSTCOMUN', 'SUPERFICIE', 'SUPERFICIETERRCOMUN',
       'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN', 'CLAVEANTERIOR', 'FECHAALTA',
       'NOMBRE', 'APATERNO', 'AMATERNO', 'CLAVECP', 'CLAVEMUNICIPIO',
       'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS', 'CLAVEASENTAMIENTO', 'EMAIL',
       'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR', 'EJERCICIO', 'CLAVEPERIODOINI',
       'CLAVEPERIODOFIN', 'VALORCATASTRAL', 'STATUS', 'No. CONSECUTIVO',
       'DOMICILIO', 'NOMBRE DEL PROPIETARIO', 'RFC', 'ESTATUS DE LA CLAVE',
       'VALOR CATASTRAL', 'SUPERFICIE DE TERRENO', 'SUPERFICIE CONSTRUCCION',
       'USO DE ÁREA HOMOGÉNEA', 'PERIODO DE RECAUDACION CORRIENTE REZAGO',
       'PERIODO DE RECAUDACION CORRIENTE', 'EJERCICIO FISCAL',
       'CLAVECATASTRAL', 'CLAVE_PREDIO', 'CLAVE_MZA', 'CLAVE_LOC'],
      dtype='object')

In [208]:
import os
import subprocess
FILEBROWSER_PATH = os.path.join(os.getenv('WINDIR'), 'explorer.exe')

def explore(path):
    # explorer would choke on forward slashes
    path = os.path.normpath(path)

    if os.path.isdir(path):
        subprocess.run([FILEBROWSER_PATH, path])
    elif os.path.isfile(path):
        subprocess.run([FILEBROWSER_PATH, '/select,', path]).__dir__
explore("C:/Users/dlara/Desktop/MUNICIPIOS/")

In [214]:
from tkinter import *                   # importing all the widgets and modules from tkinter  
from tkinter import messagebox as mb    # importing the messagebox module from tkinter  
from tkinter import filedialog as fd    # importing the filedialog module from tkinter  
import os                               # importing the os module  
import shutil   
# function to copy a file  
def copyFile():  
   # using the filedialog's askopenfilename() method to select the file  
   fileToCopy = fd.askopenfilename(  
      title = "Select a file to copy",  
      filetypes=[("All files", "*.*")]  
      )  
   # using the filedialog's askdirectory() method to select the directory  
#    directoryToPaste = fd.askdirectory(title = "Select the folder to paste the file")  
   return(fileToCopy)
#    # using the try-except method  
#    try:  
#       # using the copy() method of the shutil module to  
#       # paste the selected file to the desired directory  
#       shutil.copy(fileToCopy, directoryToPaste)  
#       # showing success message using the messagebox's showinfo() method  
#       mb.showinfo(  
#          title = "File copied!",  
#          message = "The selected file has been copied to the selected location."  
#          )  
#    except:  
#       # using the showerror() method to display error  
#       mb.showerror(  
#          title = "Error!",  
#          message = "Selected file is unable to copy to the selected location. Please try again!"  
#          )  

In [215]:
copyFile()

'C:/Users/dlara/Desktop/MUNICIPIOS/Entradas/Ixtapan_casas_cruces_todos.shp'

In [10]:
# puntos= pd.read_csv(r'C:\Users\dlara\Desktop\MUNICIPIOS\Ixtapan_puntos.csv')
pred= pd.read_csv(r'C:\Users\dlara\Downloads\Ixtapan_crucexcurtxpredial\Ixtapan_crucexcurtxpredial.csv', encoding='utf-8-sig')
pred.dropna(subset=['CLAVECATASTRAL'],inplace=True)

pred.loc[~(pred['CLAVECATASTRAL'].astype(str).str.startswith('0')),'CLAVECATASTRAL']=pred.loc[~(pred['CLAVECATASTRAL'].astype(str).str.startswith('0'))]['CLAVECATASTRAL'].astype(str).str[0:15].str.zfill(16)
pred.loc[(pred['CLAVECATASTRAL'].str.len()<16)| (pred['CLAVECATASTRAL'].str.startswith('00')),'CLAVECATASTRAL']=pred.loc[pred['CLAVECATASTRAL'].str.len()<16]['CLAVECATASTRAL'].str.ljust(16,fillchar='0')
curts=gpd.read_file(r'C:\Users\dlara\Downloads\060 Ixtapan de la Sal-20230524T163957Z-001\060 Ixtapan de la Sal\predios_060.shp')
manz=gpd.read_file(r'C:\Users\dlara\Desktop\MUNICIPIOS\Insumos_municipios_en_proceso\Ixtapan_Sal\2023 060 MZ.shp')
manz=manz.to_crs(curts.crs)
pred=pred[['DIRECCION',
       'VALORTERRENO', 'VALORTERRENOCOMUN', 'VALORCONSTPROP',
       'VALORCONSTCOMUN', 'SUPERFICIE', 'SUPERFICIETERRCOMUN',
       'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN', 'CLAVEANTERIOR', 'FECHAALTA',
       'NOMBRE', 'APATERNO', 'AMATERNO', 'CLAVECP', 'CLAVEMUNICIPIO',
       'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS', 'CLAVEASENTAMIENTO', 'EMAIL',
       'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR', 'EJERCICIO', 'CLAVEPERIODOINI',
       'CLAVEPERIODOFIN', 'VALORCATASTRAL', 'STATUS', 'No. CONSECUTIVO',
       'DOMICILIO', 'NOMBRE DEL PROPIETARIO', 'RFC', 'ESTATUS DE LA CLAVE',
       'VALOR CATASTRAL', 'SUPERFICIE DE TERRENO', 'SUPERFICIE CONSTRUCCION',
       'USO DE ÁREA HOMOGÉNEA', 'PERIODO DE RECAUDACION CORRIENTE REZAGO',
       'PERIODO DE RECAUDACION CORRIENTE', 'EJERCICIO FISCAL', 'CLAVECATASTRAL']]
pred=pred.sort_values('PERIODO DE RECAUDACION CORRIENTE', ascending=False).drop_duplicates(['CLAVECATASTRAL'])
pred['CLAVECATASTRAL']=pred['CLAVECATASTRAL'].astype(str).str.zfill(16)
pred['CLAVE_PREDIO']=pred['CLAVECATASTRAL'].str[0:10]+'000000'
pred['CLAVE_MZA']=pred['CLAVECATASTRAL'].str[0:8]+'00000000'
pred['CLAVE_LOC']=pred['CLAVECATASTRAL'].str[0:6]+'0000000000'
pred['CLAVE_LOC']=pred['CLAVE_LOC'].str.zfill(16)
pred['CLAVE_MZA']=pred['CLAVE_MZA'].str.zfill(16)
m1=pd.merge(pred, curts,  left_on='CLAVECATASTRAL', right_on='id_cat')
temp1= pred.loc[~pred['CLAVECATASTRAL'].isin(m1['CLAVECATASTRAL'])]
m2=pd.merge(temp1, curts,  left_on='CLAVE_PREDIO', right_on='id_cat')
temp2= pred.loc[(~pred['CLAVECATASTRAL'].isin(m1['CLAVECATASTRAL']))& (~pred['CLAVECATASTRAL'].isin(m2['CLAVECATASTRAL']))]
m3=pd.merge(temp2,manz,  left_on='CLAVE_MZA', right_on='cve_cat')
temp4= pred.loc[(~pred['CLAVECATASTRAL'].isin(m1['CLAVECATASTRAL']))& (~pred['CLAVECATASTRAL'].isin(m2['CLAVECATASTRAL']))& (~pred['CLAVECATASTRAL'].isin(m3['CLAVECATASTRAL']))]
m4=pd.merge(temp4,manz,  left_on='CLAVE_LOC', right_on='cve_cat')
pred_diff=pd.concat([m1,m2,m3])
pred_diff=pred_diff.sort_values('CLAVECATASTRAL')[['CLAVECATASTRAL', 'CLAVE_PREDIO', 'CLAVE_MZA', 'CLAVE_LOC', 'cve_ent',
       'cve_mun', 'cve_loc', 'id_cat', 'proceso', 'curt', 'notas', 'geometry',
       'mun', 'zona', 'manz', 'cve_cat']]


In [ ]:
pred_diff=gpd.GeoDataFrame(pred_diff, geometry='geometry', crs=4326)
igecem=pred_diff.copy()

pred_final=pd.concat([pred_diff,m4])
pred_final=pred_final.sort_values('CLAVECATASTRAL')
pred_final['LATITUD'].interpolate(method='nearest',limit_direction='backward', limit=2,inplace=True)
pred_final['LONGITUD'].interpolate(method='nearest',limit_direction='backward', limit=2,inplace=True)
pred_final=prep.transform_df_to_gpd(pred_final, 'LONGITUD','LATITUD')
pred_final=pred_final[pred_final.columns[~pred_final.columns.str.contains('cve|id|mun|zona|manz')]]
pred_final.columns=pred_final.columns.str.upper()

In [18]:
test_igecem=igecem[['CLAVE_PREDIO', 'CLAVE_MZA', 'CLAVE_LOC',  'curt','geometry']].groupby(['geometry','CLAVE_MZA'], sort=False).first().reset_index()

In [19]:
test_igecem=test_igecem.merge(pred_final.groupby('CLAVE_PREDIO').count().reset_index().sort_values(by='CLAVECATASTRAL', ascending=False)[['CLAVECATASTRAL', 'CURT', 'CLAVE_PREDIO']].rename(columns={'CLAVECATASTRAL':'ESTIMADO'}), on ='CLAVE_PREDIO')

In [20]:
test_igecem=test_igecem.sort_values(by='ESTIMADO', ascending=False)

In [54]:
try:
    puntos= prep.transform_df_to_gpd(puntos, 'LONGITUD_1','LATITUD_1', 3857)
except:
    puntos= prep.transform_df_to_gpd(puntos, 'LONGITUD','LATITUD', 4326).to_crs(3857)
    # pred=gpd.read_file(path[:path.find('_puntos.csv')] + '.shp')
# pred=pred.to_crs(3857)
puntos=puntos.set_geometry('geometry')
# pred=pred.set_geometry('geometry')
# # puntos.drop_duplicates('CLAVECATASTRAL', keep='first', inplace=True)
# puntos=puntos.groupby('geometry', sort=False, dropna=True).first().reset_index()
# try:
#     UNICOS=puntos.loc[puntos['CURT_1']<=1]
    
#     # UNICOS['geometry']=UNICOS.geometry.centroid
#     nounicos=puntos.loc[puntos['CURT_1']>1]
# except:
#     UNICOS=puntos.loc[(puntos['CURT']<=1)| (puntos['CURT'].isna())]
#     nounicos=puntos.loc[puntos['CURT']>1]
# j_puntos=prep.ckdnearest(nounicos, nounicos[['geometry']], k=2)
# try:
#     cruce_check= gpd.sjoin(j_puntos.loc[j_puntos['min_dist_2']>10][['geometry', 'LONGITUD_1', 'LATITUD_1']],pred, how='left')
# except:
#     cruce_check= gpd.sjoin(j_puntos.loc[j_puntos['min_dist_2']>10][['geometry', 'LONGITUD', 'LATITUD']],pred,how='left') 
# # cruce_check=cruce_check.loc[cruce_check['CLAVECATASTRAL'].notna()]
# cruce_check=cruce_check.groupby('geometry', as_index=False,sort=False).first().reset_index(drop=True)
# #   cruce_check.loc[cruce_check['CLAVECATAS']=='0240127004000000'].plot()
# file_name='Salidas'
# name=path[path.rfind('\\')+1:path.rfind('_puntos')]
# path12= os.path.join(path[:path.rfind('MUNICIPIOS')],'MUNICIPIOS' ,file_name)
# # cruce_check.to_file(path12 + '\\' +name+  '_predial_final_curts_reps.shp')
# # UNICOS.to_file(path12 + '\\' +name+ '_predial_final_curts_unicos.shp')
# # final_pred= pd.concat([cruce_check, UNICOS])

# filename2= glob.glob(r"C:\Users\dlara\Desktop\MUNICIPIOS\Entradas\*.shp")

    
# cruces_ads=ads_final
# print(cruces_ads.crs)
# name2=path2[path2.rfind('\\'+name)+1:path2.rfind('_cruces_todos')]

# # if cruces_ads.crs== 4326:
# #     cruces_ads=cruces_ads.to_crs(3857)
# # elif cruces_ads.crs == 3857:
# #     continue           
# # print(cruces_ads.columns)
# ### el primero es fácil porque es directo, resultado: poli 
# # gato   
# drop=UNICOS.columns[(UNICOS.columns.str.contains('left|right'))]
# UNICOS.drop(columns=drop, inplace=True)
# drop=cruces_ads.columns[(cruces_ads.columns.str.contains('left|right'))]
# cruces_ads.drop(columns=drop, inplace=True)
# primero=gpd.sjoin(cruces_ads,UNICOS, how='left' )

# ## diferenciando
# temp1=cruces_ads.loc[~cruces_ads['CURT_f'].isin(primero['CURT_f'])]
# ##cruce
# drop=temp1.columns[(temp1.columns.str.contains('left|right'))]
# temp1.drop(columns=drop, inplace=True)
# drop=cruce_check.columns[(cruce_check.columns.str.contains('left|right'))]
# cruce_check.drop(columns=drop, inplace=True)
# segundo= gpd.sjoin(temp1,cruce_check,how='left')

# sobra_pred=cruce_check[~cruce_check.geometry.isin(segundo.geometry)]
# sobra_pred.geometry=sobra_pred.geometry.buffer(2)
# real_ads= temp1.loc[~temp1['CURT_f'].isin(segundo['CURT_f'])]
# sobra_pred['CAT_1']='FUERA'
# primero['CAT_1']='PREDIAL_UNICOS'
# segundo['CAT_1']= 'PREDIAL_REPETIDOS'
# pred_final= pd.concat([primero, segundo,sobra_pred])
# pred_final.dropna(how='all', axis=1, inplace=True)
# drop=pred_final.columns[(pred_final.columns.str.contains('0|TYPES|TYPE|ADDRESS|PLUS_CODE|_GEO_|Unnamed'))]
# pred_final.drop(columns=drop, inplace=True)
# # drop2=pred_final.columns[(pred_final.columns.str.contains(r'[()\"#/@;:<>{}`+=~|.!?,]'))]
# pred_final.drop(columns=drop2, inplace=True)
# # pred_final.columns=pred_final.columns.str.replace(r"[()\"#/@;:<>{}`+=~|.!?,]", '')
# pred_final.rename(columns={'USO DE ÁREA HOMOGÉNEA':'USO DE AREA HOMOGENEA'}, inplace=True)
# # for i in pred_final.columns:
# #     if i != 'geometry':
#         # pred_final.loc[pred_final[i].astype(str).str.contains(r'[^A-Za-z0-9]+'),i]=pred_final.loc[pred_final[i].astype(str).str.contains(r'[^A-Za-z0-9]+')][i].astype(str).str.replace(r'[^A-Za-z0-9]+','')


NameError: name 'puntos' is not defined

In [ ]:
conteos=pred_final.groupby('CLAVE_PREDIO').count().reset_index().sort_values(by='CLAVECATASTRAL', ascending=False)[['CLAVECATASTRAL', 'CURT', 'CLAVE_PREDIO']].rename(columns={'CLAVECATASTRAL':'ESTIMADO'})
unicos=conteos.loc[conteos['ESTIMADO']==1]
pred_f_unicos=pred_final.loc[pred_final['CLAVE_PREDIO'].isin(unicos['CLAVE_PREDIO'])]
pred_final.loc[~pred_final['CLAVECATASTRAL'].str.endswith('000000')]
pred_final.rename(columns={'GEOMETRY':'geometry'}, inplace=True)

### Predial final

In [34]:
df_concat=task_chunks(test_igecem)

100%|██████████| 7116/7116 [13:39<00:00,  8.68it/s]


In [121]:
# df_concat.loc[df_concat['ESTIMADO']!=1]
pred_cruce=pred_final.loc[~pred_final['CLAVE_PREDIO'].isin(unicos['CLAVE_PREDIO'])]
pred_cruce.rename(columns={'GEOMETRY':'geometry'}, inplace=True)
pred_cruce=gpd.GeoDataFrame(pred_cruce, geometry='geometry',crs=4326)
df_concat=df_concat[df_concat.columns[(~df_concat.columns.isin(pred_cruce.columns)) | (df_concat.columns.astype(str).str.contains('CLAVE_PREDIO|geometry'))]]

mn1=pd.merge(pred_cruce.drop(columns=['geometry','CLAVE_PREDIO']),df_concat, left_on='CLAVECATASTRAL',right_on='CLAVE_PREDIO')
tempn1= df_concat.loc[~df_concat['CLAVE_PREDIO'].isin(mn1['CLAVECATASTRAL'])]
mn2=pd.merge(pred_cruce.drop(columns=['geometry']),tempn1, left_on='CLAVE_PREDIO',right_on='CLAVE_PREDIO')

dbfs: 
manzs= (las bases a nivel manzana) 
dbfs_final=(manzs con dbfs (nivel predio) como llave clave a nivel Manzana)


CC= debe de tener 0 al inicio y no tener ".0" al final, no debe tener más de un 0's al inicio
m1: curts con dbfs_final como llave a nivel predio ** CLAVE_PREDIO: CC.str[0:10]+'000000'
diferenciar las que no cruzan y cruzarlas a nivel manzana: CC.str[0:8]+'00000000'

In [122]:
mn1=mn1.sort_values(by=['CLAVECATASTRAL','LONGITUD_1','LATITUD_1']).drop_duplicates(subset='CLAVECATASTRAL',keep='first')
mn2=mn2.sort_values(by=['CLAVECATASTRAL','LONGITUD_1','LATITUD_1']).drop_duplicates(subset='CLAVECATASTRAL',keep='first')

In [123]:
pred_cruzado=pd.concat([mn1,mn2])

In [ ]:
nnm1=prep.ckdnearest(pred_cruce.loc[~pred_cruce['geometry'].is_empty].to_crs(3857),df_concat[['geometry','LATITUD_1','LONGITUD_1']])
# nnm1.loc[nnm1['min_dist_2']<10]
nnm1=nnm1.sort_values(by=['CLAVECATASTRAL','min_dist_2']).drop_duplicates(subset=['CLAVECATASTRAL'],keep='first')
nnm1=nnm1.loc[~nnm1['CLAVECATASTRAL'].isin(pred_cruzado['CLAVECATASTRAL'])]

In [ ]:
pred_final=pred_final.to_crs(3857)
pred_cruzado_min=pd.concat([nnm1,pred_cruzado,pred_final.loc[pred_final['CLAVE_PREDIO'].isin(unicos['CLAVE_PREDIO'])]])
pred_cruzado_min.columns[pred_cruzado_min.columns.astype(str).str.contains('CURT|curt')]

pred_cruzado['CURT']=pred_cruzado['CURT'].fillna(float('Nan'))
pred_cruzado['curt']=pred_cruzado['curt'].fillna(float('Nan'))
if all(pred_cruzado['CURT'])==all(pred_cruzado['curt']):
    pred_cruzado.drop(columns=['CURT'], inplace=True)
    
pred_cruzado_min.drop(columns=['index_left','dist',0,'NOTAS','PROCESO'], inplace=True)

pred_cruzado_minx ads 
geometry: poly


In [189]:
pred_cruzado_min

,CLAVECATASTRAL,CLAVE_PREDIO,CLAVE_MZA,CLAVE_LOC,PROCESO,CURT,NOTAS,geometry,LATITUD,LONGITUD,...,USO DE ÁREA HOMOGÉNEA,PERIODO DE RECAUDACION CORRIENTE REZAGO,PERIODO DE RECAUDACION CORRIENTE,EJERCICIO FISCAL,LATITUD_1,LONGITUD_1,min_dist_2,ORDEN,curt,ESTIMADO
0,0600101502010001,0600101502000000,0600101500000000,0600100000000000,NaN,NaN,NaN,POINT (-11095699.208 2136816.123),18.846024,-99.674362,...,NaN,NaN,NaN,NaN,2.136820e+06,-1.109571e+07,8.494446,NaN,NaN,NaN
1,0600101502010002,0600101502000000,0600101500000000,0600100000000000,NaN,NaN,NaN,POINT (-11095699.208 2136816.123),18.846024,-99.674362,...,NaN,NaN,NaN,NaN,2.136820e+06,-1.109571e+07,8.494446,NaN,NaN,NaN
2,0600101502010003,0600101502000000,0600101500000000,0600100000000000,NaN,NaN,NaN,POINT (-11095699.208 2136816.123),18.846024,-99.674362,...,NaN,NaN,NaN,NaN,2.136820e+06,-1.109571e+07,8.494446,NaN,NaN,NaN
3,0600101502010004,0600101502000000,0600101500000000,0600100000000000,NaN,NaN,NaN,POINT (-11095699.208 2136816.123),18.846024,-99.674362,...,NaN,NaN,NaN,NaN,2.136820e+06,-1.109571e+07,8.494446,NaN,NaN,NaN
4,0600101502010005,0600101502000000,0600101500000000,0600100000000000,NaN,NaN,NaN,POINT (-11095699.208 2136816.123),18.846024,-99.674362,...,NaN,NaN,NaN,NaN,2.136820e+06,-1.109571e+07,8.494446,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7061,0600913504000000,0600913504000000,0600913500000000,0600910000000000,NaN,NaN,NaN,POINT (-11098909.976 2129730.240),18.785772,-99.703205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7060,0600913506000000,0600913506000000,0600913500000000,0600910000000000,NaN,NaN,NaN,POINT (-11098909.976 2129730.240),18.785772,-99.703205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7058,0600913507000000,0600913507000000,0600913500000000,0600910000000000,NaN,NaN,NaN,POINT (-11098909.976 2129730.240),18.785772,-99.703205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7057,0600913508000000,0600913508000000,0600913500000000,0600910000000000,NaN,NaN,NaN,POINT (-11098909.976 2129730.240),18.785772,-99.703205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
pred_cruzado_min=gpd.GeoDataFrame(pred_cruzado_min, geometry='geometry')
pred_cruzado_min.to_csv('C:/Users/dlara/Ixtapan_predial_final.csv',encoding='utf-8-sig')

## Final

In [ ]:
pred_final.columns[pred_final.columns.str.contains('SUPER|VALOR|area|LS|LI')]
pred_final[['LI_95%_vc_', 'LS_95%_vc_', 'LI_95%_v_1', 'LS_95%_v_1', 'LS_95%_cat',
       'LI_95%_cat', 'PredialLI_', 'PredialLS_', 'VALORCATASTRAL',
       'VALOR CATASTRAL',
       'VALORCATAS']].dropna(how='all', axis=1).sort_values('VALORCATASTRAL')
pred_final['diff']=pred_final['LI_95%_vc_']-pred_final['VALORCATASTRAL']
pred_final['diff'].sum()

99690976.90385129

In [40]:
ix=gpd.read_file(r'C:\Users\dlara\Downloads\Atlacomulco_predial_Z\Atlacomulco_predial_Z.shp')
ix['coordenada']=ix.geometry.to_crs(4326).centroid
# ix.columns[~ix.columns.str.contains('factor|Estadist|ID|Predial|LI|LS|Media|Moda|STD|Len|Min|Max|media|min|max|Pmedia|Valor_ca')]
# t=ix[['CURT_f','clase_dete',
#        'Clase', 'Z', 'Area_Model', 'COD','COD_BV', 'COD_AH', 'Val m2_Ban',
#        'Tipo_consA', 'Frente', 'Fondo', 'Area_AH', 'Val m2_Man',  'UsoSuelo', 'VU_Terr', 'Tipo','Min_constr','Min_terren','Predialmin','geometry','coordenada']]

t=ix[['CURT_f', 'clase_dete', 'Clase', 'Area_Model', 'Induxcasa', 'tipoCenCom',
       'tipo_asent', 'nombre_act', 'Area_use','COD_BV',
       'COD_AH', 'Val m2_Ban', 'Tipo_consA', 'Frente', 'Fondo', 'Area_AH',
       'Val m2_Man','Val_m2_AH_', 'COD_Manz', 
       'VU_Terr', 'Z', 'Tipo','Min_constr','Min_terren','geometry','Predialmin','coordenada']]
# sum(ix['Min_constr']==0)
# t.info()
t['coordenada']=t['coordenada'].astype(str).str.replace('POINT', '')
# t.columns[t.columns.duplicated()]
t.to_file(r'C:\Users\dlara\Base_Adicionales_Valor_Catastral_Atlaco.shp')

In [105]:

pred_final['VALORCATASTRAL'].sum()+pred_final['VALOR CATASTRAL'].sum()-pred_final['minVC'].sum()

-28982511663.039375

In [90]:
ads_final.loc[ads_final['curt']=='3']['PredialLI_'].sum()

67711647.51311669

In [ ]:
rev_pred_t=pd.read_csv(r'C:\Users\dlara\Downloads\Ixtapan_Base_predial_PREGEO.csv', encoding='utf-8-sig')

In [ ]:
base_final=cruce_predial_adicionales()

In [ ]:
base_final.to_file(r'C:\Users\dlara\Desktop\MUNICIPIOS\Salidas\Ixtapan_terrenos_base_final_ads_pred.shp')

In [ ]:
filename2= glob.glob(r"C:\Users\dlara\Desktop\MUNICIPIOS\Entradas\*.shp")
for path2 in filename2:
    print(path2)

In [ ]:
name=path[path.rfind('\\')+1:path.rfind('_puntos')]
path12= os.path.join(path[:path.rfind('MUNICIPIOS')],'MUNICIPIOS' ,file_name)
path2[path2.rfind(path12 + '\\' +name):path2.rfind('_cruces_todos')]

In [ ]:
test0=pd.concat([m1,ADS_F,PRED_F, fin_casas], axis=0)
# .drop_duplicates('geometry')

In [ ]:
predial.loc[~predial['CLAVECATASTRAL'].isin(PRED['CLAVECATASTRAL'])]

### Distancias mínimas

In [ ]:
from shapely.geometry import Point, LineString
import geopandas as gpd
from geoloc2.preparacion_datos.src.preparacion_inter_puntos import ckdnearest
gdf=gpd.read_file("C:/Users/ASUS/Desktop/qgiszonas/points.shp")
gdf["LATLON"]=gdf["geometry"].x.astype("str")+","+gdf["geometry"].y.astype("str")
gdf


# usar primero sjoin_nearest y si hay mucho fallo usar la función ckdnearest

In [ ]:
casas=gpd.read_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\casas_t_especificios_1.shp")
casas=casas.to_crs(gdf.crs)
casas["LATLON"]=casas["geometry"].centroid.x.astype("str")+","+casas["geometry"].centroid.y.astype("str")
casas["geometry1"]=casas["geometry"]
casas["geometry"]=casas["geometry"].representative_point()
casas=casas.reset_index()
casas.rename(columns={"index":"FID"},inplace=True)
casas

In [ ]:
terreno=gpd.read_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\terreno_t_especificios_1.shp")
terreno=terreno.to_crs(gdf.crs)
terreno["LATLON"]=terreno["geometry"].centroid.x.astype("str")+","+terreno["geometry"].centroid.y.astype("str")
terreno["geometry1"]=terreno["geometry"]
terreno["geometry"]=terreno["geometry"].representative_point()
terreno

In [ ]:

# JFF_c = JFF_c.loc[JFF_c['min_dist'] < 10]
JFF_c

In [ ]:
JFF_terreno = ckdnearest(gdf.drop(columns='LATLON').to_crs(3857), terreno.to_crs(3857))
# JFF_terreno = JFF_terreno.loc[JFF_terreno['min_dist'] < 10]
JFF_terreno

In [ ]:
JFF_c["origen"]="Casas"
JFF_terreno["origen"]="Terreno"
concatenado=pd.concat([JFF_c,JFF_terreno],ignore_index=True)
concatenado.sort_values(by="min_dist",inplace=True)
concatenado=concatenado.drop_duplicates(subset=["CLAVECATAS"],keep="first")
concatenado

In [ ]:
concatenado[["FID","origen","CLAVECATAS","geometry"]].to_crs("3857").to_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\Puntos_cercanos.shp")


In [ ]:
a=concatenado[["FID","origen","CLAVECATAS","geometry1"]]
a=gpd.GeoDataFrame(a[["FID","origen","CLAVECATAS"]],geometry=a["geometry1"])
a.crs=6364
a=a.to_crs("3857")
a.to_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\Poligonos_cercanos.shp")

## Correcciones

In [ ]:
N= pd.read_csv('E:\Downloads\INTFIS_BASE_FINAL_NAUCALPAN_por_coma\INTFIS_BASE_FINAL_NAUCALPAN_por_coma.csv', encoding='utf-8-sig')

In [ ]:
PRED=N.loc[N['CLAVECATASTRAL_RECAUDACION'].notna()].dropna(how='all')

In [ ]:
correct_pred= PRED.loc[PRED['CLAVECATASTRAL'].astype(str).str.startswith('00')].groupby(['CLAVECATASTRAL','BDINTERNA_CONT_NOMBRE_COMPLETO']).first().dropna(axis=1,how='all').drop(columns='Unnamed: 0').reset_index()
# correct_pred['CLAVECATASTRAL']=correct_pred['CLAVECATASTRAL'].str[-12:].str.replace('000','', regex=False).str.zfill(10)

In [ ]:
correct_pred['CLAVECATASTRAL'].str[-16:]
correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00000'), 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00000')]['CLAVECATASTRAL'].str[-11:].str.zfill(12).str.replace('000','', regex=False).str.zfill(10)

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('000'),'CLAVECATASTRAL']=correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('000')]['CLAVECATASTRAL'].str[-16:].str.replace('000','')

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00'),'CLAVECATASTRAL'] = correct_pred.loc[correct_pred['CLAVECATASTRAL'].astype(str).str.startswith('00')]['CLAVECATASTRAL'].str[:12].str.replace('00','').str.zfill(9)

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()<9, 'CLAVECATASTRAL'] = correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()<9]['CLAVECATASTRAL'] + '00000000'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==9, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==9]['CLAVECATASTRAL'] + '0000000'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==10, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==10]['CLAVECATASTRAL'].str.zfill(10) + '000000'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==12, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==12]['CLAVECATASTRAL'] + '0000'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==13, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==13]['CLAVECATASTRAL'] + '000'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==14, 'CLAVECATASTRAL'] = correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==14]['CLAVECATASTRAL'] + '00'

In [ ]:
correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==16, 'CLAVECATASTRAL']= correct_pred.loc[correct_pred['CLAVECATASTRAL'].str.len()==16]['CLAVECATASTRAL'].str[:-6].str.replace('098','98').str.ljust(15,fillchar='0').str.zfill(16)

In [ ]:
# PRED=PRED.loc[~PRED['CLAVECATASTRAL'].astype(str).str.startswith('00')]
PRED['CLAVECATASTRAL']=PRED['CLAVECATASTRAL'].str.replace('.0','', regex=False).str.zfill(16)

In [ ]:
# PRED=PRED.dropna(axis=1,how='all').drop(columns='Unnamed: 0')
ZPRED=pd.concat([PRED,correct_pred ], axis=0)
# ZPRED.CLAVECATASTRAL.value_counts()
ZPRED.loc[ZPRED['CLAVECATASTRAL'].isna(), 'CLAVECATASTRAL'] =ZPRED.loc[ZPRED['CLAVECATASTRAL'].isna()]['CLAVECATASTRAL_RECAUDACION'].str.replace('CAT-','').astype(str)

In [ ]:
duplicados=ZPRED.loc[ZPRED['CLAVECATASTRAL'].isin(ZPRED['CLAVECATASTRAL'][ZPRED['CLAVECATASTRAL'].duplicated()])].sort_values(by='CLAVECATASTRAL').groupby(['CLAVECATASTRAL','BDINTERNA_CONT_NOMBRE_COMPLETO']).first().reset_index()

In [ ]:
ZPRED=ZPRED.loc[~ZPRED['CLAVECATASTRAL'].isin(ZPRED['CLAVECATASTRAL'][ZPRED['CLAVECATASTRAL'].duplicated()])]
ZZPRED= pd.concat([ZPRED,duplicados], axis=0)

In [ ]:
ZZPRED['CLAVECATASTRAL_RECAUDACION'] = 'CAT-' + ZZPRED['CLAVECATASTRAL']

In [ ]:
NF=pd.read_csv(r'C:\Users\dlara\Downloads\drive-download-20230607T215848Z-001\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv',encoding='utf-8-sig')

In [ ]:
NFS=gpd.read_file(r'C:\Users\dlara\Downloads\drive-download-20230607T215848Z-001\BASE_MEJORADA_NAUCALPAN_URLS_v2.shp')

In [ ]:
PRED2=NF.loc[NF['CLAVECATASTRAL'].notna()]

In [ ]:
ZZPRED.loc[ZZPRED['CLAVECATASTRAL']=='09807751100E0904']

In [ ]:
PRED2['CLAVECATASTRAL']=PRED2['CLAVECATASTRAL'].astype(str).str.replace('.0','', regex=False).str.zfill(16)

In [ ]:
PRED2['CLAVECATASTRAL_RECAUDACION'] = 'CAT-' + PRED2['CLAVECATASTRAL']

In [ ]:

# denue=PRED2.columns[PRED2.columns.str.contains('DENUE')]
# PRED2.drop(columns=denue,inplace=True)
# drop1=PRED2.columns[(~PRED2.columns.isin(ZZPRED.columns)) & (PRED2.columns.str.contains('SAT|ESTATUS|NOMINA'))]
# PRED2.drop(columns=drop1,inplace=True)

drop2=PRED2.columns[(~PRED2.columns.isin(ZZPRED.columns)) & (PRED2.columns.str.contains('N_EXT|KEY|GRUPO_CONTROL|BDINTERNA|IMSS|CALLE|COLONIA|ADDRESS|PLUS_CODE|_GEO_|Unnamed'))]
PRED2.drop(columns=drop2,inplace=True)

In [ ]:
PRED2[['CLAVECATASTRAL','LATLON',  'CVEGEO',
       'Tipo Polígono', 'ORIGEN', 'TIPO DE COORDENADA', 'CURT_NOTAS', 'IMPORTE_FEB',
       'IMPORTE_MAR', 'ACTIVO', 'ACTIVIDAD_ECONOMICA', 'NUM_TRABAJADORES',
       'RANGO_TRABAJADORES', 'CLASIFICACION_TRABAJADORES', 'ACTIVO2',
       'CANTIDAD_ANIOS_ADEUDOS', 'ZONA_CATASTRAL', 'IMPORTE_FEB_X_ANIO',
       'CLASIFICACION_VALORCATASTRAL', 'USO', 'INTFIS_RS_FD_COOR_RECAUDACION',
       'INTFIS_RS_FD_ESTADO', 'INTFIS_RS_FD_PAIS', 'INTFIS_RS_FD_URLS',
       'CONSECUTIVO', 'RFC', 'RFC_CONTEOS', 'INTFIS_RS_FD_FULL_DOM', 'ID_CAT',
       'curt', 'DOMICILIO', 'NUM_EXT', 'NUM_INT', 'CP', 'CONST_DOM',
       'DOM_UNICOS', 'PERSONA', 'VALORCATASTRAL_2008', 'VALORCATASTRAL_2009',
       'VALORCATASTRAL_2010', 'VALORCATASTRAL_2011', 'VALORCATASTRAL_2012',
       'VALORCATASTRAL_2013', 'VALORCATASTRAL_2014', 'VALORCATASTRAL_2015',
       'PREDIAL_PROPIETARIO_EMAIL', 'PREDIAL_PROPIETARIO_CURP',
       'PREDIAL_PROPIETARIO_RFC', 'IMPORTETOTAL_ACUMULATIVO_AGUA',
       'CANTIDAD_NIS', 'NIS', 'CANTIDAD_CLAVES_CATASTRALES',
       'CLAVES_CATASTRALES', 'INTFIS_RS_FD_LATLON','curt','CURT','INTFIS_RS_FD_LON', 'INTFIS_RS_FD_LAT','INTFIS_RS_FD_LON_1', 'INTFIS_RS_FD_LAT_1','INTFIS_RS_FD_LAT_2', 'INTFIS_RS_FD_LON_2', '0', 'geometry']]

In [ ]:
ZZPRED.loc[~ZZPRED['CLAVECATASTRAL'].isin(PRED2['CLAVECATASTRAL'])]
# PRED2.loc[~PRED2['CLAVECATASTRAL'].isin(ZZPRED['CLAVECATASTRAL'])]